In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import seaborn as sns
from IPython.display import display, Markdown
az.style.use('arviz-darkgrid')
np.random.seed(44)


In [2]:
import seaborn as sns
sns.set_theme()
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 'medium'
plt.rcParams.update({
    "figure.figsize": [8, 4],
    'figure.facecolor': '#fffff8',
    'axes.facecolor': '#fffff8',
    'figure.constrained_layout.use': True,
    'font.size': 14.0,
    'hist.bins': 'auto',
    'lines.linewidth': 3.0,
    'lines.markeredgewidth': 2.0,
    'lines.markerfacecolor': 'none',
    'lines.markersize': 8.0, 
})
sns.set(rc={'figure.figsize':(8,4)})

# Generalized Linear Models


Giorgio Corani <br/>
*Bayesian Data Analysis and Probabilistic Programming*
<br/>
<br/>
``giorgio.corani@supsi.ch``





# Credits 


*  Mostly based on Chapter 4 of O. Martin, *Bayesian Analysis with Python, Second Edition*.

*  Notebook by G. Corani

# Outline

* So far, we assumed the response variable to be Gaussian distributed, whose mean is given by a linear combination of the independent variables.  

* We now remove the Gaussian assumption to obtain:

    * Logistic regression
    * Poisson regression
    * Zero-inflated Poisson regression

## Generalized linear models

* Main idea:  predict the mean of a response variable as  a linear combination of independent variables:
$$\mu = f(\alpha + X \beta) $$

* $f$ is the *inverse link function*, since previous literature was applying the function to the left-hand side of the equation.

# Inverse Link function

$$\mu = f(\alpha + X \beta)$$


* The simplest inverse link function is  the identity function, which returns the same value used as the argument.

*  By using the identity function, we obtain linear regression.


## Regresión de Poisson

Otro modelo lineal generalizado muy popular es la regresión de Poisson. Este modelo asume que los datos se distribuyen de acuerdo con la distribución de Poisson.

Un escenario en el que la distribución de Poisson es útil es cuando se analizan cosas, como la descomposición de un núcleo radioactivo, el número de hijos por pareja o el número de seguidores de Twitter. Lo que todos estos ejemplos tienen en común es que usualmente los modelamos usando números discretos no negativos {0, 1, 2, 3 ...}. Este tipo de variable recibe el nombre de datos de conteo (count data).

### La distribución de Poisson

Imagina que estamos contando la cantidad de autos rojos que pasan por una avenida por hora. Podríamos usar la distribución de Poisson para describir estos datos. La distribución de Poisson se utiliza generalmente para describir la probabilidad que ocurra un número determinado de eventos independientes entre si en un intervalo de tiempo o espacio fijo. Esta distribución discreta se parametriza utilizando solo un valor, $\mu$ (la tasa, también comúnmente representada con la letra griega $\lambda$). $\mu$ corresponde a la media y también a la varianza de la distribución. La función de probabilidad de masa de la distribución de Poisson es:

$$ f(x \mid \mu) = \frac {e^{-\mu}\mu^x} {x!} \tag{4.17}$$

dónde:
* $\mu$ es el número promedio de eventos por unidad de tiempo / espacio
* $x$ es un valor entero positivo 0, 1, 2, ...
* $x!$ es el factorial de x, k! = k × (k - 1) × (k - 2) × ... × 2 × 1

En la siguiente gráfica, podemos ver algunos ejemplos de la familia de distribución de Poisson, para diferentes valores de $\mu$.

In [ ]:
mu_params = [0.5, 1.5, 3, 8]
x = np.arange(0, max(mu_params) * 3)
for mu in mu_params:
    y = stats.poisson(mu).pmf(x)
    plt.plot(x, y, 'o-', label=f'μ = {mu:3.1f}')
plt.legend()
plt.xlabel('x')
plt.ylabel('f(x)');

Es importante notar que $\mu$ puede ser un flotante, pero la distribución modela probabilidad de un número discreto de eventos. En la figura 4.10, los puntos representan los valores de la distribución, mientras que las líneas continuas son una ayuda visual que nos ayuda a comprender fácilmente la _forma_ de la distribución. Recuerde, la distribución de Poisson es una distribución discreta.

La distribución de Poisson puede verse como un caso especial de la distribución binomial cuando la cantidad de intentos $n$ es muy grande pero la probabilidad de éxito $p$ es muy baja. Sin entrar en detalles matemáticos, tratemos de aclarar la afirmación anterior. Siguiendo el ejemplo del auto, podemos afirmar que o vemos el auto rojo o no, por lo que podemos usar una distribución binomial. En ese caso tenemos:

$$ x \sim Bin(n, p) \tag{4.18}$$

Entonces, la media de la distribución binomial es:

$$\mathbf{E}[x] = np \tag{4.19} $$

Y la varianza viene dada por:

$$ \mathbf {V}[x] = np (1 - p) \tag{4.20}$$

Pero tenga en cuenta que incluso si se encuentra en una avenida muy transitada, la posibilidad de ver un auto rojo en comparación con el número total de automóviles en una ciudad es muy pequeño y, por lo tanto, tenemos:

$$n >> p \Rightarrow np \simeq np (1-p) \tag{4.21}$$

Entonces, podemos hacer la siguiente aproximación:

$$\mathbf {V}[x] = np \tag{4.22}$$

Ahora la media y la varianza están representadas por el mismo número y podemos
declarar con confianza que nuestra variable se distribuye como una distribución de Poisson:

$$x \sim Poisson(\mu = np) \tag{4.23}$$

## El modelo de Poisson inflado de ceros

Al contar cosas, una posibilidad es no contar esas cosas, es decir obtener cero. El número cero puede ocurrir generalmente por muchas razones; obtuvimos un cero porque estábamos contando autos rojos y un auto rojo no pasó por la avenida o porque no logramos verlo (tal vez no vimos pasar un diminuto auto rojo detrás de un gran camión). Entonces, si usamos una distribución de Poisson, notaremos, por ejemplo, cuando realizamos una verificación predictiva posterior, que el modelo generó menos ceros en comparación con los datos.

¿Cómo arreglamos eso? Podemos tratar de abordar la causa exacta por la cual nuestro modelo predice menos ceros de los observados e incluir ese factor en el modelo. Sin embargo, suele ser el caso, que es suficiente y más fácil para nuestro propósito, asumir que simplemente tenemos una mezcla de dos procesos:

* Uno modelado por una distribución de Poisson con probabilidad $\psi$
* Otra persona que da ceros adicionales con probabilidad $1 - \psi$.

Esto se conoce como modelo Poisson inflado de ceros (ZeroInflatedPoisson). En algunos textos, encontrarás que $\psi$ se usa para representar los ceros extra y $1-\psi$ la probabilidad de Poisson.

Básicamente una distribución ZIP nos dice que:

$$p(y_j = 0) = 1 - \psi + (\psi) e^{-\mu} \tag{4.24}$$

$$p(y_j = k_i ) = \psi \frac{\mu^x_i e^{-\mu}}{x_i!} \tag{4.25}$$ 

Donde $1-\psi$ es la probabilidad de ceros adicionales. Podríamos implementar fácilmente estas ecuaciones en un modelo PyMC3. Sin embargo, podemos hacer algo aún más fácil y usar la distribución ZIP de PyMC3.

In [ ]:
#np.random.seed(42)
n = 100
θ_real = 2.5
ψ = 0.1

# Simulate some data
counts = np.array([(np.random.random() > (1-ψ)) * np.random.poisson(θ_real)
                   for i in range(n)])

In [ ]:
with pm.Model() as ZIP:
    ψ = pm.Beta('ψ', 1., 1.)
    θ = pm.Gamma('θ', 2., 0.1)
    y = pm.ZeroInflatedPoisson('y', ψ, θ, observed=counts)
    trace = pm.sample(1000)

In [ ]:
az.plot_trace(trace);

In [ ]:
az.summary(trace)

## Regresión de Poisson y regresión ZIP

El modelo ZIP puede parecer un poco aburrido, pero a veces necesitamos estimar distribuciones simples como esta u otra como las distribuciones de Poisson o Gaussianas. Además, podemos usar las distribuciones Poisson o ZIP como parte de un modelo lineal. Como vimos con la regresión logística (y softmax) podemos usar una función de enlace inverso para transformar el resultado de un modelo lineal en una variable adecuada para ser utilizada con otra distribución que no sea la normal. En la siguiente figura, vemos una posible implementación de una regresión ZIP. La regresión de Poisson será similar, pero sin la necesidad de incluir $\phi$ ya que no modelaremos un exceso de ceros. Observe que ahora usamos la función exponencial como la función de enlace inverso. Esta elección garantiza que los valores devueltos por el modelo lineal sean positivos.

Para ejemplificar la implementación de un modelo de regresión ZIP, vamos a trabajar con un conjunto de datos tomado del [Instituto de Investigación y Educación Digital](http://www.ats.ucla.edu/stat/data).

El problema es el siguiente: trabajamos en la administración de un parque y queremos mejorar la experiencia de los visitantes. Por lo tanto, decidimos realizar una breve encuesta a 250 grupos que visitan el parque. Parte de los datos que recopilamos (a nivel de grupo) consiste en:

* La cantidad de peces que capturaron (contar)
* Cuántos niños había en el grupo (niño)
* Ya sea que hayan traído o no una casa-rodante o "caravana" al parque (camper).

Usando estos datos, vamos a construir un modelo que predice el número de peces capturados en función de las variables niño y caravana. Podemos usar Pandas para cargar los datos:

In [ ]:
fish_data = pd.read_csv('datos/fish.csv')

Lo dejo como un ejercicio para que explore el conjunto de datos utilizando gráficos y / o una función de Pandas, como `describe()`. Por ahora vamos a continuar traduciendo el diagrama de Kruschke anterior a PyMC3:

In [ ]:
with pm.Model() as ZIP_reg:
    ψ = pm.Beta('ψ', 1, 1)
    α = pm.Normal('α', 0, 10)
    β = pm.Normal('β', 0, 10, shape=2)
    θ = pm.math.exp(α + β[0] * fish_data['child'] + β[1] * fish_data['camper'])
    yl = pm.ZeroInflatedPoisson('yl', ψ, θ, observed=fish_data['count'])
    trace_ZIP_reg = pm.sample(1000)
az.plot_trace(trace_ZIP_reg);

Para entender mejor los resultados de nuestra inferencia, hagamos una gráfica.

In [ ]:
children = [0, 1, 2, 3, 4]
fish_count_pred_0 = []
fish_count_pred_1 = []
for n in children:
    without_camper = trace_ZIP_reg['α'] + trace_ZIP_reg['β'][:,0] * n
    with_camper = without_camper + trace_ZIP_reg['β'][:,1]
    fish_count_pred_0.append(np.exp(without_camper))
    fish_count_pred_1.append(np.exp(with_camper))
    
    
plt.plot(children, fish_count_pred_0, 'C0.', alpha=0.01)
plt.plot(children, fish_count_pred_1, 'C1.', alpha=0.01)

plt.xticks(children);
plt.xlabel('Number of children')
plt.ylabel('Fish caught')
plt.plot([], 'C0o', label='without camper')
plt.plot([], 'C1o', label='with camper')
plt.legend();

## Regresión logística robusta

Acabamos de ver cómo corregir un exceso de ceros sin modelar directamente el factor que los genera. Se puede utilizar un enfoque similar, sugerido por Kruschke, para realizar una versión más robusta de la regresión logística. Recuerde que en la regresión logística modelamos los datos como binomiales, es decir, ceros y unos. Por lo tanto, puede suceder que encontremos un conjunto de datos con ceros y/o unos inusuales. Tomemos como ejemplo el conjunto de datos de iris que ya hemos visto, pero con algunos _datos intrusos_ agregados de manera deliberada:

In [ ]:
iris = sns.load_dataset("iris") 
df = iris.query("species == ('setosa', 'versicolor')") 
y_0 = pd.Categorical(df['species']).codes 
x_n = 'sepal_length'  
x_0 = df[x_n].values 
y_0 = np.concatenate((y_0, np.ones(6, dtype=int))) 
x_0 = np.concatenate((x_0, [4.2, 4.5, 4.0, 4.3, 4.2, 4.4])) 
x_c = x_0 - x_0.mean() 
plt.plot(x_c, y_0, 'o', color='k');

Aquí tenemos algunas versicolors (1s) con una longitud de sépalo inusualmente corta. Podemos arreglar esto con un modelo de mezcla. Vamos a decir que la variable de salida viene con probabilidad $\pi$ por adivinación aleatoria o con probabilidad $1-\pi$ de un modelo de regresión logística. Matemáticamente, tenemos:

$$p = \pi \ 0.5 + (1 - \pi) \: \text{logistic}(\alpha + X \beta) \tag{4.26} $$


Tenga en cuenta que cuando $\pi = 1$ obtenemos $p = 0.5 $, y para $\pi = 0 $ recuperamos la expresión para regresión logística.

La implementación de este modelo es una modificación directa del primer modelo de este capítulo.

In [ ]:
with pm.Model() as modelo_rlg:
    α = pm.Normal('α', mu=0, sd=10)
    β = pm.Normal('β', mu=0, sd=10)
    
    μ = α + x_c *  β  
    θ = pm.Deterministic('θ', pm.math.sigmoid(μ))
    bd = pm.Deterministic('bd', -α/β)
    
    π = pm.Beta('π', 1, 1) 
    p = π * 0.5 + (1 - π) * θ 
    
    yl = pm.Bernoulli('yl', p=p, observed=y_0)

    trace_rlg = pm.sample(1000)

In [ ]:
az.summary(trace_rlg, varnames)

In [ ]:
theta = trace_rlg['θ'].mean(axis=0)
idx = np.argsort(x_c)
plt.plot(x_c[idx], theta[idx], color='C2', lw=3);
plt.vlines(trace_rlg['bd'].mean(), 0, 1, color='k')
bd_hpd = pm.hpd(trace_rlg['bd'])
plt.fill_betweenx([0, 1], bd_hpd[0], bd_hpd[1], color='k', alpha=0.5)

plt.scatter(x_c, np.random.normal(y_0, 0.02), marker='.', color=[f'C{x}' for x in y_0])
theta_hpd = pm.hpd(trace_rlg['θ'])[idx]
plt.fill_between(x_c[idx], theta_hpd[:,0], theta_hpd[:,1], color='C2', alpha=0.5)

plt.xlabel(x_n)
plt.ylabel('θ', rotation=0)
# use original scale for xticks
locs, _ = plt.xticks() 
plt.xticks(locs, np.round(locs + x_0.mean(), 1));

## Ejercicios

1. Vuelva a correr el `modelo_0` pero esta vez usando las variables `petal_length` y `petal_width` ¿En que difieren los resultados? ¿Cuán ancho o angosto es el intervalo hpd 94%? 

2. Repita el ejercicio 1, esta vez usando una distribución t de Student como _prior ligeramente informativo_. Pruebe con distintos valores de $\nu$.

3. Use un modelo lineal (como los vistos en el capítulo anterior) para  clasificar setosa o versicolor en función de `sepal_length`. ¿Cuán útil es este modelo comparado con una regresión logística? 

4. En la sección _Interpretando los coeficientes de una regresion logística_ vimos el efecto sobre el `log_odds` de cambiar la variable `sepal_length` en 1 unidad. Usando la figura 4.6 corrobore que el valor obtenido para `log_odds_versicolor_i` se corresponde con el valor de `probability_versicolor_i`. Haga lo mismo para `log_odds_versicolor_f` y `probability_versicolor_f`. Si solo sabemos que el valor de `log_odds_versicolor` es negativo que podemos decir de la probabilidad de versicolor, use la figura 4.6 como guía ¿Es este resultado evidente de la definición de log-odds?

5. Para `modelo_1` verifica cuanto cambian el valor de log-odd al incrementar `sepal_leght` de 5.5 a 6.5. ¿Cúal es el cambio en valores de probabilidad? ¿Cuál es el cambio en términos de log-odds y probabilidad al pasar de 4.5 a 5.5?

6. En el ejemplo de clases desbalanceadas cambie `df = df[45:]` por `df = df[22:78]`. Esto dejará más o menos el mismo número de datos, pero con las clases balanceadas. Compare con los resultados previos. ¿Cuál caso es más parecido a usar el conjunto de datos completo?

7. Suponga que en vez de usar una regresión softmax usamos un modelo lineal simple codificando $\text{setosa}=0$, $\text{versicolor}=1$ y $\text{virginica}=1$. Bajo el modelo lineal simple que pasaría si cambiáramos el orden del código.

8. Compara los likelihoods para el `modelo_0` y para el `modelo_lda`. Usa la función `pm.sample_posterior_predictive` para generar datos a partir de estos dos modelos. ¿En que difirien los datos predichos para ambos modelos?

9. Extienda el modelo `ZIP_reg` para incluir la variable `persons`. Usa esta variable para modelar el número de ceros extra. Deberás obtener un modelo que incluya dos modelos lineales, uno que conecte las variables `children` y `camper` a la tasa de Poisson y otro que conecte el número de personas con la variable $\psi$. Presta atención si es necesario usar una función inversa de enlace.

10. Use los datos empleados en el ejemplo de la regresión logística robusta con un modelo de regresión logística simple. ¿Cuál es el efecto de los _outliers_? Pruebe agregando o eliminado _outliers_.